In [ ]:
import pandas as pd
import numpy as np
import altair as alt
!pip install geopandas
import geopandas as gpd
!pip install cenpy
import cenpy as cen
from getpass import getpass

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [ ]:
# Read in data of ALL banks in the country in 2020. Ran this through Google Maps API separately and have coords

df = pd.read_csv("ALL_2020.csv", encoding="ISO-8859-1", engine="python")
pd.set_option('display.max_columns', None)

In [ ]:
df['ZIPBR'] = df['ZIPBR'].apply(str)

# Concat address columns together

df['ADDRESS_FULL'] = df['ADDRESBR'].str.cat(
    df['CITYBR'],sep=", ").str.cat(
    df['STALPBR'],sep=", ").str.cat(
    df['ZIPBR'],sep=", ")

In [ ]:
#df.head(2)

In [ ]:
CENSUS_API_KEY = getpass('Enter your Census API Key: ')

Enter your Census API Key: ··········


In [ ]:
# I'm searching for population and income per county. 

availableTables = cen.explorer.available()
print(f"The API offers access to {len(availableTables)} datasets")

The API offers access to 653 datasets


In [ ]:
acs = cen.products.ACS()

In [ ]:
acs.filter_tables('POPULATION', by='description')

,description,columns
table_name,,
B00001,UNWEIGHTED SAMPLE COUNT OF THE POPULATION,[B00001_001E]
B01003,TOTAL POPULATION,[B01003_001E]
B05006,PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION...,"[B05006_001E, B05006_002E, B05006_003E, B05006..."
B05007,PLACE OF BIRTH BY YEAR OF ENTRY BY CITIZENSHIP...,"[B05007_001E, B05007_002E, B05007_003E, B05007..."
B05008,SEX BY PLACE OF BIRTH BY YEAR OF ENTRY FOR THE...,"[B05008_001E, B05008_002E, B05008_003E, B05008..."
...,...,...
C24030,SEX BY INDUSTRY FOR THE CIVILIAN EMPLOYED POPU...,"[C24030_001E, C24030_002E, C24030_003E, C24030..."
C24040,"SEX BY INDUSTRY FOR THE FULL-TIME, YEAR-ROUND ...","[C24040_001E, C24040_002E, C24040_003E, C24040..."
C24050,INDUSTRY BY OCCUPATION FOR THE CIVILIAN EMPLO...,"[C24050_001E, C24050_002E, C24050_003E, C24050..."


In [ ]:
acs.filter_variables('B01003')

,label,concept,predicateType,group,limit,predicateOnly,attributes,required,values
B01003_001E,Estimate!!Total,TOTAL POPULATION,int,B01003,0,NaN,"B01003_001EA,B01003_001M,B01003_001MA",NaN,NaN


In [ ]:
con = cen.remote.APIConnection('ACSDT5Y2019',apikey=CENSUS_API_KEY)
variables = con.variables
# Total Population and Median Household Income
columns = [
    'B01003_001', # Total Population
    'B19013_001', # Median Household Income
]
# Include margins of error in estimate 
columns_E = [i+'E' for i in columns]
columns_M = [i+'M' for i in columns]

g_unit = 'county'

income = con.query(columns_E + columns_M, geo_unit=g_unit)

In [ ]:
# 5 digit FIPS is 2 digit state + 3 digit county

income['fips_code'] = income['state'].str.cat(
    income['county'])

In [ ]:
income = income.rename(columns={"B01003_001E": "POPULATION", 
                         "B19013_001E": "MEDIAN_HOUSEHOLD_INCOME", 
                         "fips_code": "FIPS"})

In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [ ]:
# Using crosswalk to add Name and State to census data

fips = pd.read_csv("fips_codes.csv")

In [ ]:
fips['FIPS'] = fips['FIPS'].apply(pd.to_numeric)
income['FIPS'] = income['FIPS'].apply(pd.to_numeric)
fips.head()

,FIPS,Name,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL


In [ ]:
#income = income.drop(['B01003_001M', 'B19013_001M'], axis = 1)
income_fips = fips.merge(income, on='FIPS', how='left')
income_fips.head()

,FIPS,Name,State,POPULATION,MEDIAN_HOUSEHOLD_INCOME,B01003_001M,B19013_001M,state,county
0,1001,Autauga,AL,55380,58731,-555555555,4410,01,001
1,1003,Baldwin,AL,212830,58320,-555555555,1564,01,003
2,1005,Barbour,AL,25361,32525,-555555555,2291,01,005
3,1007,Bibb,AL,22493,47542,-555555555,5504,01,007
4,1009,Blount,AL,57681,49358,-555555555,2136,01,009


In [ ]:
# Selecting relevant columns from dataframe of all banks

banks = df[['NAMEFULL', 'CNTYNAMB', 'STALPBR', 'SIMS_LATITUDE', 'SIMS_LONGITUDE','ASSET', 'BKCLASS', 'ADDRESS_FULL']].copy()
banks = banks.rename(columns={"CNTYNAMB": "Name", 
                         "STALPBR": "State"})
banks.head(2)

,NAMEFULL,Name,State,SIMS_LATITUDE,SIMS_LONGITUDE,ASSET,BKCLASS,ADDRESS_FULL
0,State Street Bank and Trust Company,Suffolk,MA,42.352692,-71.057967,"276,808,000",SM,"1 Lincoln St. Fl 1, Boston, MA, 2111"
1,State Street Bank and Trust Company,Mercer,NJ,40.346826,-74.593563,"276,808,000",SM,"600 College Road East, Princeton, NJ, 8540"


In [ ]:
# Merge banks with fips+income, so I can now see which county each bank is in, 
# which FIPS code that corresponds with, 
# and what the population and median household income is in that county. 

# Merged on 2 criteria to account for repeating county names

merged_banks = pd.merge(income_fips, banks,  how='left', left_on=['Name','State'], right_on = ['Name','State'])

In [ ]:
# Spot checking to make sure common county names in different states are assigned the right FIPS code
merged_banks[merged_banks.Name == 'Washington'].head()

,FIPS,Name,State,POPULATION,MEDIAN_HOUSEHOLD_INCOME,B01003_001M,B19013_001M,state,county,NAMEFULL,SIMS_LATITUDE,SIMS_LONGITUDE,ASSET,BKCLASS,ADDRESS_FULL
1393,1129,Washington,AL,16541,41370,-555555555,3437,01,129,"The First, A National Banking Association",31.615291,-88.370991,"5,075,725",N,"30122 Hwy 17 South, Millry, AL, 36558"
1394,1129,Washington,AL,16541,41370,-555555555,3437,01,129,"The First, A National Banking Association",31.496954,-88.277053,"5,075,725",N,"34 Court St, Chatom, AL, 36518"
1395,1129,Washington,AL,16541,41370,-555555555,3437,01,129,Smartbank,31.266630,-88.031204,"3,265,878",SM,"158 Commerce Street, Mcintosh, AL, 36553"
1396,1129,Washington,AL,16541,41370,-555555555,3437,01,129,Smartbank,31.464369,-88.254409,"3,265,878",SM,"16780 Jordan Street, Chatom, AL, 36518"
3825,5143,Washington,AR,232289,50451,-555555555,1049,05,143,Generations Bank,36.084617,-94.170474,"602,380",NM,"3665 N Investment Drive, Fayetteville, AR, 72703"


Analysis: 

GOALS: 
1. **DONE:** Count number of banks per head per FIPS code (Count number of entries divided by population)
2. **DONE:** Use Equifax county level credit data to look at trends in credit scores in the 10. 
3. Use Safegraph 2018 data to check how frequented these branches were. Check foot-traffic of other branches in the FIPS code region. 
4. Last resort: Pick the lowest ones and do spot analysis on which banks closed between 2018 and 2020. Branch data for 2018 here: https://www7.fdic.gov/sod/dynaDownload.asp?barItem=6

Note: I also have data on mergers in case this could become useful: https://www.fdic.gov/bank/individual/merger/2020/2020.pdf

In [ ]:
# Number of banks per FIPS code

FIPS_counts = merged_banks.FIPS.value_counts().rename_axis('FIPS').to_frame('Banks')

# Merge this with population data -- dataframe "income"

FIPS_counts = FIPS_counts.merge(income, on='FIPS', how='left')
FIPS_counts.head()

# Divide Banks by POPULATION 
  
FIPS_counts = FIPS_counts.dropna()
FIPS_counts['POPULATION'] = FIPS_counts['POPULATION'].astype(int)
FIPS_counts['BANKS_PER_CAP*100'] = (FIPS_counts['Banks']/FIPS_counts['POPULATION'])*100

In [ ]:
# Creating a df called banks_cap 
# banks_cap is just FIPS_counts sorted by banks per capita and income, 
# Merged in with fips codes. 

banks_cap = FIPS_counts.sort_values(by=["BANKS_PER_CAP*100", "MEDIAN_HOUSEHOLD_INCOME"]).merge(fips, on="FIPS", how='left')
banks_cap.head(10)

,FIPS,Banks,POPULATION,MEDIAN_HOUSEHOLD_INCOME,B01003_001M,B19013_001M,state,county,BANKS_PER_CAP*100,Name,State
0,29189,1,996919,67420,-555555555,683,29,189,0.000100,St Louis,MO
1,17043,1,929060,92809,-555555555,1225,17,043,0.000108,Du Page,IL
2,24033,1,908670,84920,-555555555,802,24,033,0.000110,Prince Georges,MD
3,13089,1,749323,62399,-555555555,935,13,089,0.000133,De Kalb,GA
4,11001,1,692683,86420,-555555555,1008,11,001,0.000144,Washington,DC
5,24510,1,609032,50379,-555555555,820,24,510,0.000164,Baltimore City,MD
6,51810,1,450201,76610,-555555555,1188,51,810,0.000222,Virginia Beach City,VA
7,29183,1,394290,84978,-555555555,1195,29,183,0.000254,St Charles,MO
8,12111,1,312947,52322,-555555555,1459,12,111,0.000320,St Lucie,FL
9,29510,1,308174,43896,-555555555,1098,29,510,0.000324,St Louis City,MO


Here's my credit score data: https://fred.stlouisfed.org/searchresults/?st=subprime%20population&t=minnehaha%20county%2C%20sd&rt=minnehaha%20county%2C%20sd&ob=sr

From this, counties with the FEWEST banks per head and their credit score data: (REDOING THIS WITH 3 BANKS MINIMUM)
1. St Louis, MO: https://fred.stlouisfed.org/series/EQFXSUBPRIME029189 
2. Du Page, IL: https://fred.stlouisfed.org/series/EQFXSUBPRIME017043
3. Prince Georges, MD: https://fred.stlouisfed.org/series/EQFXSUBPRIME024033
4. De Kalb, GA: https://fred.stlouisfed.org/series/EQFXSUBPRIME013089 
5. Washington, DC: https://fred.stlouisfed.org/series/EQFXSUBPRIME011001 
6. Baltimore City, MD: https://fred.stlouisfed.org/series/EQFXSUBPRIME024510 
7. Virgina Beach City, VA: https://fred.stlouisfed.org/series/EQFXSUBPRIME051810 
8. St Charles, MO: https://fred.stlouisfed.org/series/EQFXSUBPRIME029183 
9. St Lucie, FL: https://fred.stlouisfed.org/series/EQFXSUBPRIME012111 
10. St Louis City, MO: https://fred.stlouisfed.org/series/EQFXSUBPRIME029510 

Lowest:(REDOING THIS WITH 50K POP MIN)
1. Towner, ND: https://fred.stlouisfed.org/series/EQFXSUBPRIME038095 
2. Petroleum, MT: https://fred.stlouisfed.org/series/EQFXSUBPRIME030069 
3. Blaine, NE: https://fred.stlouisfed.org/series/EQFXSUBPRIME031009 

Analysis here: 
My lowest ones still have low credit scores, and this is definitely because I have a mix of urban, rural and small and large. Need to do this again but set a minimum population -- 50,000 is reasonable, to make sure I'm not just looking at outliers for the "Lowest" category. 

**STEPS**
1. Redo lowest using 50,000 as population
2. Once complete, manually add in average credit score 2020 data for highest 10 and lowest 10. 

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='bankscredit.csv')  
banks_cap.to_csv('bankscredit.zip', index=False,
          compression=compression_opts)

Analysis from credit score and income data:

CREDIT SCORES
1. In the 10 counties with the lowest number of banks per capita, on average, 32% of people have a credit score under 660. 

2. Meanwhie, in the 10 counties with the most banks per capita, only 24% have credit scores under 660. 

INCOME
1. In the 10 counties with the lowest number of banks per capita, average household income is $54,674. 

2. In the 10 counties with the most banks per capita, average household income is $60170